In [93]:
import json
import os
import numpy as np
import pandas as pd

In [ ]:
json_results_file = "results.json"
problems_list = ['LOGHAIRY',
                 'MARATOSB',
                 'TOINTGOR',
                 'SENSORS',
                 'DIXMAANL',
                 'DIXMAANB',
                 'DIXMAANI1',
                 'ARWHEAD',
                 'POWELLSG',
                 'SCHMVETT',
                 'INDEF',
                 'SCURLY20',
                 'BOX',
                 'SCURLY10',
                 'SPARSQUR'
                ]

all_method_names = ["BFGS", "SA", "G(5)", "G(10)", "NLSA"]
all_step_methods = ["CS", "BB1", "BB2"]

In [101]:
def read_json_to_dict(file_path):
    data = {}

    if os.path.exists(file_path):
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

    return data

In [102]:
all_results = read_json_to_dict(json_results_file)
for problem_name in list(all_results.keys()):
    all_results[problem_name.replace("\n", "")] = all_results.pop(problem_name)

In [106]:
times = []
values = []
method_names_columns = []

for i, problem_name in enumerate(problems_list):
    if problem_name in list(all_results.keys()):
        moving_execution_times = []
        moving_execution_values = []
        for method_name in [name for name in list(all_results[problem_name].keys()) if name != "size"]:
            for step_size_method_name in [name for name in list(all_results[problem_name][method_name])]:
                if i == 0:
                    method_names_columns.append(f"{method_name} - {step_size_method_name}")
                actual_mean = np.mean([value['execution_time'] for value in all_results[problem_name][method_name][step_size_method_name]])
                actual_value = np.min([value['value'] for value in all_results[problem_name][method_name][step_size_method_name]])
                actual_iterations = np.mean([value['iterations'] for value in all_results[problem_name][method_name][step_size_method_name]])

                moving_execution_times.append(actual_mean)
                moving_execution_values.append(actual_value)

        
        min_times_indices = np.where(moving_execution_times == min(moving_execution_times))[0]
        min_values_indices = np.where(moving_execution_values == min(moving_execution_values))[0]

        moving_execution_times.insert(0, [method_names_columns[index] for index in min_times_indices])
        moving_execution_values.insert(0, [method_names_columns[index] for index in min_values_indices])

        moving_execution_times.insert(0, problem_name.replace("\n", ""))
        moving_execution_values.insert(0, problem_name.replace("\n", ""))


        times.append(moving_execution_times)
        values.append(moving_execution_values)

method_names_columns.insert(0, "Problem's name")
method_names_columns.insert(1, "Best")

df_times = pd.DataFrame(times, columns=method_names_columns)
df_values = pd.DataFrame(values, columns=method_names_columns)

In [107]:
df_times

,Problem's name,Best,BFGS - Default,SA - CS,SA - BB1,SA - BB2,G(10) - CS,G(10) - BB1,G(10) - BB2,G(5) - CS,G(5) - BB1,G(5) - BB2,NLSA - CS,NLSA - BB1,NLSA - BB2
0,LOGHAIRY,[G(5) - BB1],0.001479,0.076579,0.000892,0.001673,0.088377,0.000938,0.001827,0.082460,0.000890,0.001588,0.089968,0.000999,0.001873
1,MARATOSB,[BFGS - Default],0.064791,0.465676,0.178111,0.101796,0.835474,0.083195,0.091822,0.738858,0.161124,0.101640,0.643333,0.112093,0.094447
2,TOINTGOR,[NLSA - BB2],0.021557,0.057115,0.016190,0.015874,0.113694,0.017091,0.015918,0.075753,0.016533,0.016184,0.068900,0.016324,0.014770
3,SENSORS,[BFGS - Default],0.056137,0.425620,0.151801,0.144255,16.916796,0.134438,0.089823,3.584704,0.151577,0.086385,2.923298,0.136661,0.087052
4,DIXMAANL,[BFGS - Default],0.051015,0.286006,0.121299,0.224598,12.768384,0.072367,0.141023,8.244355,0.078534,0.143960,12.593118,0.104206,0.143006
5,DIXMAANB,[G(10) - BB2],0.018374,0.011360,0.013534,0.013670,0.025632,0.010664,0.008315,0.016557,0.010840,0.010210,0.060183,0.009616,0.010710
6,DIXMAANI1,[G(10) - BB2],0.077160,0.838865,0.150341,0.122279,0.521296,0.104449,0.059612,0.518251,0.093895,0.060498,0.527965,0.083895,0.064648
7,ARWHEAD,[G(10) - BB1],0.027956,8.306173,0.022963,0.023571,11.095093,0.017676,0.019759,10.184404,0.018061,0.018664,8.526199,0.020524,0.019796
8,POWELLSG,[BFGS - Default],0.045412,4.744211,0.550625,0.088662,4.009777,0.070586,0.062357,4.101594,0.128244,0.061129,3.948804,0.064213,0.065959


In [108]:
df_values

,Problem's name,Best,BFGS - Default,SA - CS,SA - BB1,SA - BB2,G(10) - CS,G(10) - BB1,G(10) - BB2,G(5) - CS,G(5) - BB1,G(5) - BB2,NLSA - CS,NLSA - BB1,NLSA - BB2
0,LOGHAIRY,"[SA - BB2, G(10) - BB2, G(5) - BB2, NLSA - BB2]",6.533607e+00,6.542226e+00,6.551658e+00,6.432048e+00,6.542226e+00,6.551658e+00,6.432048e+00,6.542226e+00,6.551658e+00,6.432048e+00,6.542226e+00,6.551658e+00,6.432048e+00
1,MARATOSB,[G(5) - BB1],-1.000000e+00,9.958825e-01,-8.944888e-01,-9.999996e-01,9.958826e-01,-1.000000e+00,-9.999998e-01,9.958825e-01,-1.000000e+00,-9.999999e-01,9.958825e-01,-9.999997e-01,-9.999996e-01
2,TOINTGOR,[G(10) - CS],1.373905e+03,1.373905e+03,1.373905e+03,1.373905e+03,1.373905e+03,1.373905e+03,1.373905e+03,1.373905e+03,1.373905e+03,1.373905e+03,1.373905e+03,1.373905e+03,1.373905e+03
3,SENSORS,"[G(10) - BB2, G(5) - BB2, NLSA - BB2]",-2.108531e+03,-2.108531e+03,-2.106000e+03,-2.109375e+03,-2.095875e+03,-2.106000e+03,-2.109375e+03,-2.095875e+03,-2.106000e+03,-2.109375e+03,-2.101781e+03,-2.106000e+03,-2.109375e+03
4,DIXMAANL,[BFGS - Default],1.000298e+00,2.640183e+00,2.600680e+00,2.523439e+00,2.257505e+03,2.626949e+00,2.561769e+00,1.679543e+01,2.600686e+00,2.561769e+00,2.265679e+03,2.473259e+00,2.561769e+00
5,DIXMAANB,[G(10) - CS],1.000010e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
6,DIXMAANI1,[SA - BB2],1.011262e+00,1.001115e+00,1.001317e+00,1.000999e+00,1.001115e+00,1.001288e+00,1.001006e+00,1.001115e+00,1.001253e+00,1.001006e+00,1.001115e+00,1.001320e+00,1.001370e+00
7,ARWHEAD,"[SA - BB2, G(10) - BB2, G(5) - BB2, NLSA - BB2]",6.459096e-09,3.353313e-08,1.554001e-11,4.440004e-12,4.864024e-09,1.554001e-11,4.440004e-12,5.182595e-09,1.554001e-11,4.440004e-12,1.519647e-08,1.554001e-11,4.440004e-12
8,POWELLSG,[G(5) - BB2],5.741271e-04,7.371069e-03,1.738849e-04,1.096120e-04,7.355262e-03,1.304351e-04,7.903812e-05,7.366081e-03,1.435522e-04,6.610822e-05,7.381377e-03,1.297207e-04,7.028460e-05


In [113]:

all_best_times = [value for value in df_times["Best"]]
all_best_values = [value for value in df_values["Best"]]

In [114]:
all_best_times

[['G(5) - BB1'],
 ['BFGS - Default'],
 ['NLSA - BB2'],
 ['BFGS - Default'],
 ['BFGS - Default'],
 ['G(10) - BB2'],
 ['G(10) - BB2'],
 ['G(10) - BB1'],
 ['BFGS - Default']]

In [115]:
all_best_values

[['SA - BB2', 'G(10) - BB2', 'G(5) - BB2', 'NLSA - BB2'],
 ['G(5) - BB1'],
 ['G(10) - CS'],
 ['G(10) - BB2', 'G(5) - BB2', 'NLSA - BB2'],
 ['BFGS - Default'],
 ['G(10) - CS'],
 ['SA - BB2'],
 ['SA - BB2', 'G(10) - BB2', 'G(5) - BB2', 'NLSA - BB2'],
 ['G(5) - BB2']]